In [1]:
!pip install catboost
!pip install pycaret
!pip install optuna
!pip install wandb

In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from pycaret.regression import *

import wandb
wandb.init(project="DACON_235877")

seed=1011
cv = 5
top_n_model=5
tune_iter=10

# def set_seeds(seed=seed):
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     random.seed(seed)
#     np.random.seed(seed)

# set_seeds()

wandb.log({'seed': seed, 'cv': cv, 'top_n_model': top_n_model, 'tune_iter': tune_iter})

train = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/data/test.csv")

train.head()

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,id,Gender,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Target
0,1,M,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,15
1,2,I,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,8
2,3,I,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,18
3,4,M,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,13
4,5,I,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,6


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1253 non-null   int64  
 1   Gender          1253 non-null   object 
 2   Lenght          1253 non-null   float64
 3   Diameter        1253 non-null   float64
 4   Height          1253 non-null   float64
 5   Whole Weight    1253 non-null   float64
 6   Shucked Weight  1253 non-null   float64
 7   Viscra Weight   1253 non-null   float64
 8   Shell Weight    1253 non-null   float64
 9   Target          1253 non-null   int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 98.0+ KB


### Train

In [4]:
s = setup(train, target='Target',
          fold_strategy='kfold', fold=cv,
          session_id=seed, log_experiment=True)

,Description,Value
0,session_id,1011
1,Target,Target
2,Original Data,"(1253, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(877, 10)"


* Blending

In [5]:
%%time
blender = blend_models(compare_models(sort='MAE', n_select=top_n_model), choose_better=True)
tuned_blender = tune_model(blender, n_iter=tune_iter, optimize='MAE', search_library='optuna', choose_better=True)
final_model=finalize_model(tuned_blender)
final_model

MAE     MSE    RMSE      R2   RMSLE    MAPE
Split  Fold                                                
CV-Val 0     1.7433  5.6639  2.3799  0.4420  0.1953  0.1752
       1     1.5256  5.1587  2.2713  0.4843  0.1698  0.1430
       2     1.4598  3.9645  1.9911  0.6157  0.1715  0.1537
       3     1.5341  5.0222  2.2410  0.4995  0.1825  0.1543
       4     1.5193  4.5860  2.1415  0.5035  0.1976  0.1515
       Mean  1.5564  4.8791  2.2050  0.5090  0.1833  0.1555
       Std   0.0970  0.5722  0.1312  0.0576  0.0116  0.0106
Train  NaN   1.4279  3.9487  1.9871  0.6038  0.1664  0.1434

CPU times: user 22.8 s, sys: 1.27 s, total: 24.1 s
Wall time: 1min 14s


* Stacking

In [6]:
# %%time
# stacker = stack_models(compare_models(sort='MAE', n_select=top_n_model), choose_better=True)
# tuned_stacker = tune_model(stacker, n_iter=tune_iter, optimize='MAE', search_library='optuna', choose_better=True)
# final_model=finalize_model(tuned_stacker)
# final_model

### Test

In [7]:
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [8]:
# save_model(final_model, 'my_best_pipeline')
# loaded_model = load_model('my_best_pipeline')
# print(loaded_model)

In [9]:
predictions = predict_model(final_model, data=test)
predictions.shape

(2924, 10)

In [10]:
submission=predictions['Label'].round()

sample_submission = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/data/sample_submission.csv")
sample_submission.Target = submission
sample_submission.to_csv("/content/drive/MyDrive/DACON-Basic/235877_전복 나이 예측 경진대회/submission.csv",index=False)